In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the CSV file containing the User IDs and URLs
df_csv = pd.read_csv('User_Review_url_and_ID.csv')

# Headers to simulate a browser
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "cookie": 'ccsid=997-0059822-9974992; locale=en; blocking_sign_in_interstitial=true; csm-sid=448-9044706-8238887; session-id=138-9449735-4650719; lc-main=en_US; ubid-main=132-0831767-7645715; csm-hit=tb:s-HH6TEBC541XWDPX2HW08|1728400665806&t:1728400665807; session-id-time=2359120669l; session-token="9epd7M6HsVRPHteGVvJu5Fk8d+KvASerta8tz+la5w+TVREtncxVnfcCzs8K9EpSg+fQrgx1GgnsD2rLlayR4DmjBe1MsI/hzqbpLxW9fzltLac/Bx4GP/iysuf1f336QEHPYkFM5wvwKAJRlJF/gUtvoqUNgeHO7d0nUgwzQ4n10cVaWcJP8kU3S8U/EG5f1z4kuWhJEC/GxOGAFPM2AwR1aAfDTGc3oUmUkI4heyVFL2PVr5DhINiO23tre2UqOAtOSgArJ6foDhvdkRwB1A2cwygZeAAPu6znpJ+cxdwCK+Qw0zKdF5L4dt5MPkikWWxVqpvSWD4krg+mGhL2EJhFdAQbaWn2SM7qiWrwNN8="; x-main="qY09NsWCzSNW8b933Yonj4af6qs5TDB0DUsYapgS18@nyKFkHdJj5p@Mt2dldE6F"; at-main=Atza|IwEBIKKe6dZi9mkGtv2CtS3qsv9NuaJNSNpmNpDSxRXvt17a_x2-zjEpiAOHkEQ34JuOVngoGfAl8dIqliYqOigT30w9CGWKtZI6rgDWz0mruXs-yNgr_nsxBWgaXid-1e8ZvmgQryKo5OgAIx-Lce0ocNWWSfh3g77KeZNETFdonWPAuJz5V82UuuKblmr-o03ZzeyBC7rbWUbFLRhtcSWy7Mqogs8ONE5dM3XgdBITj5b5F2h4Z_ih2WgCM4wTps3r6HY; sess-at-main="l6swvzfG2thASaG98K8Nu/j0D/+teOfY5a1UhpsZBz8="; likely_has_account=true; *session*id2=4cd4cabfc8c16e891c0f62348aa076dd',
    "if-none-match": 'W/"862d60fdc6915eed33d69a373690f36d"',
    "priority": "u=0, i",
    "sec-ch-ua": '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

# Initialize a DataFrame to store ratings
ratings_data = []

# Iterate through each User_ID in the CSV file
for index, row in df_csv.iterrows():
    user_id = row['User_ID']
    print(f"Processing User ID: {user_id}")
    
    for i in range(1, 4):  # Loop through pages if needed, modify the range accordingly
        print(f"Scraping page {i}")
        url = f"https://www.goodreads.com/review/list/{user_id}?order=d&page={i}&sort=rating&view=reviews"
        print(f"Scraping URL: {url}")
        
        # Request the page
        response = requests.get(url, headers=headers)
       # print(response)
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all rows in the table where book ID and rating are stored
        #rows = soup.find_all('tr', class_='bookalike review')
        rows = soup.find_all('tr', class_='bookalike review')
        #print(rows)

        for row in rows:
            # Step 1: Find the <td> with the book ID in the link
            book_td = row.find('td', class_='field title')
            #print(book_td)
            if book_td:
                bb = book_td.find('div', class_='value')
                # print(bb)
                a_tag = bb.find('a', href=True)
               # print(a_tag)
                if a_tag and '/book/show/' in a_tag['href']:
                    book_url = a_tag['href']
                    book_id = book_url.split('/book/show/')[1].split('-')[0].split('.')[0]
                    print(book_id)
                    # Step 2: Find the <td> with the rating (sibling or close td)
                    rating_td = row.find('td', class_='field rating')
                    if rating_td:
                        rating_span = rating_td.find('span', class_='staticStars')
                        if rating_span:
                           # rating = rating_span['title']  # Extract the rating from 'title'
                            stars = rating_td.find_all('span', class_='staticStar p10')
                            print('rating')
                            print(len(stars))# Append the data to the list
                            rating=len(stars)
                            ratings_data.append({
                                'User_ID': user_id,
                                'Book_ID': book_id,
                                'Rating': rating
                            })

# Convert to DataFrame
ratings_df = pd.DataFrame(ratings_data)

# Display or save the DataFrame
print(ratings_df)

# Optionally, save to a CSV file
ratings_df.to_csv('scraped_ratings.csv', index=False)

# Convert to DataFrame
ratings_df = pd.DataFrame(ratings_data)

# Display or save the DataFrame
print(ratings_df)

# Optionally, save to a CSV file
#ratings_df.to_csv('scraped_ratings.csv', index=False) can u explain this part with example rating_td = div.find_next_sibling('td', class_='field rating')

Processing User ID: 14008088
Scraping page 1
Scraping URL: https://www.goodreads.com/review/list/14008088?order=d&page=1&sort=rating&view=reviews
14116
rating
5
14118
rating
5
24812
rating
5
72915
rating
5
72910
rating
5
76234
rating
5
77174
rating
5
7356990
rating
5
87420
rating
5
140130
rating
5
146122
rating
5
146128
rating
5
166353
rating
5
166356
rating
5
166396
rating
5
166399
rating
5
179173
rating
5
9602495
rating
5
22446777
rating
5
13498737
rating
5
Scraping page 2
Scraping URL: https://www.goodreads.com/review/list/14008088?order=d&page=2&sort=rating&view=reviews
22620436
rating
5
250131
rating
5
250135
rating
5
79121
rating
5
132391
rating
5
604208
rating
5
583569
rating
5
24523250
rating
5
646056
rating
5
647934
rating
5
661854
rating
5
670666
rating
5
670672
rating
5
2495797
rating
5
53316316
rating
5
20956687
rating
5
23454027
rating
5
12664136
rating
5
670686
rating
5
11136560
rating
5
Scraping page 3
Scraping URL: https://www.goodreads.com/review/list/14008088?order=d&